# ICH Toy Simulation – Dark Energy from Influx/Outflux Imbalance

**Logic Realism Theory (LRT) + Information Circulation Hypothesis (ICH) toy model**

February 2026 – iterative development

## Core Idea

Late-time cosmic acceleration emerges from a small persistent imbalance:
**influx from I∞ > outflux via black holes** in the logical circulation cycle.

### Features
- Entropy-dependent influx (self-regulating feedback)
- Emergent BH formation, absorption, Hawking-like leak, merging
- Comoving BH positions
- Friedmann-like expansion from imbalance
- Distance modulus μ(z) + residuals vs real anchors (z=0.44, z=1.0)
- χ² goodness-of-fit

### LRT Context
- **I∞**: Infinite logical possibility space (source of influx)
- **AΩ**: Actualized instantiations (particles in simulation)
- **Black holes**: Recycling mechanism returning instantiations to I∞
- **Imbalance**: Net positive influx drives Λ-like acceleration

Run all cells in order. Adjust parameters in the first code cell.

In [ ]:
# =============================================
#  PARAMETERS – tune these!
# =============================================

import numpy as np
import random
import matplotlib.pyplot as plt
from collections import deque
from scipy.interpolate import interp1d
from scipy.stats import chi2

# Simulation control
N_STEPS = 20000
UNIVERSE_SIZE_0 = 80.0   # Comoving size
ACCEPT_PROB = 0.08       # Probability of accepting an influx attempt

# Initial conditions: start at high redshift
A_INITIAL = 0.25         # Start at a=0.25 (z=3), evolve toward a=1 (z=0)

# Influx (entropy-dependent)
INFLUX_PEAK = 0.25       # Max influx rate at low entropy
INFLUX_FLOOR = 0.02      # Min influx rate at high entropy
ENTROPY_SENSITIVITY_K = 3.0  # How strongly entropy suppresses influx

ENTROPY_DRIFT = 0.0006   # Per-step entropy increase for particles
CLUSTER_PULL = 0.06      # Gravitational clustering strength

# Black holes
BH_ABSORB_RADIUS = 6     # Comoving radius for absorption
BH_SHRINK_PER_PROCESS = 0.5  # Mass lost per absorption event
BH_HAWKING_LEAK = 0.003  # Hawking evaporation rate
BH_MIN_MASS = 2.0        # Below this, BH evaporates completely
BH_FORM_WINDOW = 15      # Spatial window for density check
BH_FORM_DENSITY_TH = 0.5 # Density threshold for BH formation
BH_FORM_ENTROPY_MAX = 0.6  # Max local entropy for BH formation
BH_FORM_PROB = 0.03      # Probability of BH forming when conditions met
BH_INITIAL_MASS_NEW = 30 # Initial mass of newly formed BH
BH_MERGE_COMOVING_DIST = 12.0  # Distance threshold for merging
BH_MERGE_PROB = 0.35     # Probability of merge when close
BH_MERGE_EFFICIENCY = 0.92  # Mass retention in merger

# Cosmology - CRITICAL: H0 controls expansion rate
H0 = 0.00005             # Initial Hubble parameter (much smaller for slower expansion)
DT = 1.0                 # Time step
OMEGA_M0 = 0.30          # Matter density parameter today
IMBALANCE_COUPLING = 0.002  # How strongly imbalance affects expansion (reduced)
DILUTION_POWER = 3.2     # Dilution exponent for ρ_Λ_eff

# w_eff smoothing
W_EFF_EMA_ALPHA = 0.08   # EMA smoothing for equation of state

# Distance modulus & χ²
REF_Z_POINTS = [0.44, 1.0]   # Reference redshifts (SNe Ia anchors)
REF_MU_VALUES = [42.8, 44.3] # Expected μ at those redshifts
REF_SIGMA = [0.18, 0.20]     # Uncertainties
Z_INTEGRATION_POINTS = 100   # Points for numerical integration
Z_LOW_NORMALIZATION = 0.05   # Low-z normalization point
TARGET_MU_LOWZ = 36.5        # Target μ at low-z

## State Initialization

In [ ]:
# Particle list: each particle has {pos, entropy, age}
particles = []

# Black hole list: each BH has {pos, mass, active}
black_holes = []

# Scale factor and Hubble - START AT HIGH REDSHIFT
a = A_INITIAL  # Start at z=3 (a=0.25)
volume_comoving = UNIVERSE_SIZE_0
H = H0

# Counters
total_instantiated = 0
total_recycled = 0
total_bh_formed = 0
cumulative_imbalance = 0.0

# History for plotting
history = {
    "step": [], "a": [], "H": [], "rho_L_eff": [], "imbalance": [],
    "n_particles": [], "n_active_bh": [], "recycled": [],
    "avg_entropy": [], "influx_rate": [], "w_eff": [],
    "z": [], "mu_simulated": [], "mu_residuals": []
}

# For acceleration calculation
da_dt_history = deque(maxlen=5)
a_history = deque(maxlen=5)
smoothed_d2a_dt2 = 0.0
smoothed_w_eff = -1.0

## Helper Functions

In [ ]:
def physical_volume():
    """Physical volume = comoving volume × scale factor."""
    return volume_comoving * a


def compute_avg_entropy():
    """Average entropy of all particles."""
    if not particles:
        return 0.0
    return np.mean([p["entropy"] for p in particles])


def get_influx_rate(avg_entropy):
    """
    Entropy-dependent influx rate.
    High entropy suppresses influx (self-regulation).
    """
    suppression = 1 + ENTROPY_SENSITIVITY_K * avg_entropy
    return INFLUX_FLOOR + (INFLUX_PEAK - INFLUX_FLOOR) / suppression


def add_new_particles(avg_entropy):
    """
    Influx from I∞: add new particles based on current conditions.
    Returns (number_added, current_influx_rate).
    """
    global total_instantiated
    rate = get_influx_rate(avg_entropy)
    n_attempts = int(physical_volume() * rate)
    added = 0
    for _ in range(n_attempts):
        if random.random() < ACCEPT_PROB:
            pos = random.uniform(0, UNIVERSE_SIZE_0)
            particles.append({
                "pos": pos,
                "entropy": 0.04 + random.random() * 0.22,
                "age": 0
            })
            added += 1
            total_instantiated += 1
    return added, rate


def evolve_particles():
    """
    Evolve particles: increase entropy, age, apply clustering.
    """
    if not particles:
        return
    
    # Get BH positions for clustering
    bh_positions = [bh["pos"] for bh in black_holes if bh["active"]]
    
    for p in particles:
        # Entropy drift
        p["entropy"] += ENTROPY_DRIFT
        p["age"] += 1
        
        # Gravitational pull toward nearest BH
        if bh_positions:
            # Find nearest BH (with periodic boundary)
            min_dist = float('inf')
            nearest_bh_pos = None
            for bh_pos in bh_positions:
                dist = abs(p["pos"] - bh_pos)
                dist = min(dist, UNIVERSE_SIZE_0 - dist)  # Periodic
                if dist < min_dist:
                    min_dist = dist
                    nearest_bh_pos = bh_pos
            
            if nearest_bh_pos is not None and min_dist > 0:
                # Move toward BH
                direction = 1 if nearest_bh_pos > p["pos"] else -1
                # Handle periodic wrap
                if abs(p["pos"] - nearest_bh_pos) > UNIVERSE_SIZE_0 / 2:
                    direction *= -1
                pull = CLUSTER_PULL / (1 + min_dist * 0.1)
                p["pos"] += direction * pull
                p["pos"] = p["pos"] % UNIVERSE_SIZE_0

In [ ]:
def absorb_and_evaporate():
    """
    Black holes absorb nearby particles and evaporate via Hawking.
    Returns number of particles recycled.
    """
    global total_recycled
    recycled_this_step = 0
    
    for bh in black_holes:
        if not bh["active"]:
            continue
        
        # Find particles within absorption radius
        to_remove = []
        for i, p in enumerate(particles):
            dist = abs(p["pos"] - bh["pos"])
            dist = min(dist, UNIVERSE_SIZE_0 - dist)  # Periodic
            if dist < BH_ABSORB_RADIUS:
                to_remove.append(i)
                bh["mass"] += 1  # Gain mass from absorption
        
        # Remove absorbed particles (reverse order to preserve indices)
        for i in reversed(to_remove):
            particles.pop(i)
            recycled_this_step += 1
            total_recycled += 1
        
        # Shrink from processing
        if to_remove:
            bh["mass"] -= BH_SHRINK_PER_PROCESS
        
        # Hawking evaporation
        bh["mass"] -= BH_HAWKING_LEAK
        
        # Check if evaporated
        if bh["mass"] < BH_MIN_MASS:
            bh["active"] = False
    
    return recycled_this_step


def check_and_merge_black_holes():
    """
    Merge nearby black holes probabilistically.
    """
    active_bhs = [bh for bh in black_holes if bh["active"]]
    if len(active_bhs) < 2:
        return
    
    merged_indices = set()
    
    for i in range(len(active_bhs)):
        if i in merged_indices:
            continue
        for j in range(i + 1, len(active_bhs)):
            if j in merged_indices:
                continue
            
            bh1, bh2 = active_bhs[i], active_bhs[j]
            dist = abs(bh1["pos"] - bh2["pos"])
            dist = min(dist, UNIVERSE_SIZE_0 - dist)
            
            if dist < BH_MERGE_COMOVING_DIST and random.random() < BH_MERGE_PROB:
                # Merge: larger absorbs smaller
                if bh1["mass"] >= bh2["mass"]:
                    bh1["mass"] += bh2["mass"] * BH_MERGE_EFFICIENCY
                    bh2["active"] = False
                else:
                    bh2["mass"] += bh1["mass"] * BH_MERGE_EFFICIENCY
                    bh1["active"] = False
                merged_indices.add(j)

In [ ]:
def try_form_black_hole():
    """
    Check if conditions are right for spontaneous BH formation.
    """
    global total_bh_formed
    
    if not particles:
        return
    
    # Sample a random position
    center = random.uniform(0, UNIVERSE_SIZE_0)
    
    # Count particles in window
    local_particles = []
    for p in particles:
        dist = abs(p["pos"] - center)
        dist = min(dist, UNIVERSE_SIZE_0 - dist)
        if dist < BH_FORM_WINDOW:
            local_particles.append(p)
    
    if not local_particles:
        return
    
    # Check density threshold
    local_density = len(local_particles) / (2 * BH_FORM_WINDOW)
    if local_density < BH_FORM_DENSITY_TH:
        return
    
    # Check entropy threshold
    local_entropy = np.mean([p["entropy"] for p in local_particles])
    if local_entropy > BH_FORM_ENTROPY_MAX:
        return
    
    # Probabilistic formation
    if random.random() < BH_FORM_PROB:
        black_holes.append({
            "pos": center,
            "mass": BH_INITIAL_MASS_NEW,
            "active": True
        })
        total_bh_formed += 1

In [ ]:
def update_cosmology(added, recycled, avg_entropy):
    """
    Update scale factor and Hubble parameter based on imbalance.
    Returns (rho_L_eff, imbalance, w_eff).
    """
    global a, H, cumulative_imbalance, smoothed_d2a_dt2, smoothed_w_eff
    
    # Imbalance: influx - outflux
    imbalance = added - recycled
    cumulative_imbalance += imbalance
    
    # Effective dark energy density from cumulative imbalance
    # Dilutes slower than matter (power < 3)
    rho_L_eff = IMBALANCE_COUPLING * cumulative_imbalance / (a ** DILUTION_POWER) if a > 0 else 0
    
    # Matter density (dilutes as a^-3)
    rho_m = OMEGA_M0 / (a ** 3) if a > 0 else OMEGA_M0
    
    # Total energy density
    rho_total = rho_m + max(rho_L_eff, 0)
    
    # Friedmann equation: H² ∝ ρ_total
    H = H0 * np.sqrt(rho_total / (OMEGA_M0 + 0.001)) if rho_total > 0 else H0 * 0.1
    
    # Store for acceleration calculation
    da_dt = H * a
    da_dt_history.append(da_dt)
    a_history.append(a)
    
    # Update scale factor
    a += H * a * DT
    
    # Compute w_eff from acceleration
    w_eff = smoothed_w_eff  # Default to previous
    if len(da_dt_history) >= 3:
        # Numerical second derivative
        d2a_dt2 = (da_dt_history[-1] - 2*da_dt_history[-2] + da_dt_history[-3]) / (DT**2)
        smoothed_d2a_dt2 = W_EFF_EMA_ALPHA * d2a_dt2 + (1 - W_EFF_EMA_ALPHA) * smoothed_d2a_dt2
        
        if rho_L_eff > 1e-10 and a > 0:
            # w_eff from acceleration equation
            # ä/a = -4πG/3 * (ρ + 3p) → w = p/ρ
            # For acceleration: w < -1/3
            w_raw = -1 - (2 * smoothed_d2a_dt2 * a) / (3 * H**2 * a**2 + 1e-10)
            w_eff = max(-2.5, min(-0.3, w_raw))  # Clamp to reasonable range
            smoothed_w_eff = W_EFF_EMA_ALPHA * w_eff + (1 - W_EFF_EMA_ALPHA) * smoothed_w_eff
    
    return rho_L_eff, imbalance, smoothed_w_eff

In [ ]:
def get_H_at_z(z, H_interp):
    """
    Get Hubble parameter at redshift z using interpolation.
    """
    if z < H_interp.x.min():
        return H_interp(H_interp.x.min())
    elif z > H_interp.x.max():
        return H_interp(H_interp.x.max())
    return H_interp(z)


def compute_comoving_distance(z_target, H_interp):
    """
    Compute comoving distance to redshift z_target.
    d_c = c ∫_0^z dz'/H(z')
    """
    if z_target <= 0:
        return 0.0
    
    z_vals = np.linspace(0, z_target, Z_INTEGRATION_POINTS)
    integrand = [1.0 / max(get_H_at_z(z, H_interp), 1e-10) for z in z_vals]
    d_c = np.trapezoid(integrand, z_vals)
    return d_c


def compute_luminosity_distance(z, H_interp):
    """
    Luminosity distance: d_L = (1 + z) * d_c
    """
    d_c = compute_comoving_distance(z, H_interp)
    return (1 + z) * d_c

In [ ]:
def normalize_distance_modulus():
    """
    Post-process: compute μ(z) for all recorded points.
    Normalize to match low-z anchor.
    """
    # Build H(z) interpolator from history
    z_hist = np.array(history["z"])
    H_hist = np.array(history["H"])
    
    # Sort by z for interpolation
    sort_idx = np.argsort(z_hist)
    z_sorted = z_hist[sort_idx]
    H_sorted = H_hist[sort_idx]
    
    # Remove duplicates and zeros
    mask = (z_sorted > 0) & (H_sorted > 0)
    z_sorted = z_sorted[mask]
    H_sorted = H_sorted[mask]
    
    if len(z_sorted) < 2:
        print("Warning: Not enough valid z-H pairs for interpolation")
        return
    
    # Remove duplicate z values
    _, unique_idx = np.unique(z_sorted, return_index=True)
    z_sorted = z_sorted[unique_idx]
    H_sorted = H_sorted[unique_idx]
    
    H_interp = interp1d(z_sorted, H_sorted, kind='linear', fill_value='extrapolate')
    
    # Compute raw μ for all points
    mu_raw = []
    for z in history["z"]:
        if z > 0:
            d_L = compute_luminosity_distance(z, H_interp)
            if d_L > 0:
                mu = 5 * np.log10(d_L) + 25  # Standard distance modulus
            else:
                mu = np.nan
        else:
            mu = np.nan
        mu_raw.append(mu)
    
    # Find normalization offset at low-z
    mu_at_lowz = None
    for i, z in enumerate(history["z"]):
        if abs(z - Z_LOW_NORMALIZATION) < 0.01 and not np.isnan(mu_raw[i]):
            mu_at_lowz = mu_raw[i]
            break
    
    if mu_at_lowz is None:
        # Find closest to low-z
        valid_idx = [(i, abs(z - Z_LOW_NORMALIZATION)) for i, z in enumerate(history["z"]) 
                     if not np.isnan(mu_raw[i])]
        if valid_idx:
            best_idx = min(valid_idx, key=lambda x: x[1])[0]
            mu_at_lowz = mu_raw[best_idx]
    
    offset = TARGET_MU_LOWZ - mu_at_lowz if mu_at_lowz else 0
    
    # Apply normalization
    history["mu_simulated"] = [m + offset if not np.isnan(m) else np.nan for m in mu_raw]
    
    # Compute residuals vs ΛCDM expectation (simplified)
    # Using approximate ΛCDM: μ ≈ 5 log10(z) + 42.4 + 2.5 log10(1+z) for z > 0.1
    history["mu_residuals"] = []
    for i, z in enumerate(history["z"]):
        if z > 0.05 and not np.isnan(history["mu_simulated"][i]):
            # Approximate ΛCDM
            mu_lcdm = 5 * np.log10(z * (1 + z) / H0) + 25
            mu_lcdm_normalized = mu_lcdm + offset
            residual = history["mu_simulated"][i] - mu_lcdm_normalized
        else:
            residual = np.nan
        history["mu_residuals"].append(residual)

In [ ]:
def compute_chi2_goodness_of_fit():
    """
    Compute χ² goodness-of-fit to reference SNe Ia points.
    """
    chi2_val = 0.0
    n_points = 0
    
    for ref_z, ref_mu, sigma in zip(REF_Z_POINTS, REF_MU_VALUES, REF_SIGMA):
        # Find closest simulated point
        best_idx = None
        best_dist = float('inf')
        for i, z in enumerate(history["z"]):
            mu_val = history["mu_simulated"][i]
            # Check for valid mu value
            if mu_val is not None and not (isinstance(mu_val, float) and np.isnan(mu_val)):
                dist = abs(z - ref_z)
                if dist < best_dist:
                    best_dist = dist
                    best_idx = i
        
        if best_idx is not None and best_dist < 0.1:
            mu_sim = history["mu_simulated"][best_idx]
            chi2_val += ((mu_sim - ref_mu) / sigma) ** 2
            n_points += 1
    
    dof = n_points - 1 if n_points > 1 else 1
    chi2_reduced = chi2_val / dof if dof > 0 else np.nan
    p_value = 1 - chi2.cdf(chi2_val, dof) if n_points > 0 else np.nan
    
    # Interpretation
    if np.isnan(chi2_reduced):
        interpretation = "No valid data points for fit"
    elif chi2_reduced < 1.5:
        interpretation = "Good fit (χ²/dof < 1.5)"
    elif chi2_reduced < 3.0:
        interpretation = "Acceptable fit (χ²/dof < 3)"
    else:
        interpretation = "Poor fit (χ²/dof >= 3)"
    
    return {
        "chi2": chi2_val,
        "dof": dof,
        "chi2_reduced": chi2_reduced,
        "p_value": p_value,
        "interpretation": interpretation
    }

## Run the Simulation

In [ ]:
# Reset state for fresh run
particles = []
black_holes = []
a = A_INITIAL  # Start at high redshift
H = H0
total_instantiated = 0
total_recycled = 0
total_bh_formed = 0
cumulative_imbalance = 0.0
da_dt_history.clear()
a_history.clear()
smoothed_d2a_dt2 = 0.0
smoothed_w_eff = -1.0

history = {
    "step": [], "a": [], "H": [], "rho_L_eff": [], "imbalance": [],
    "n_particles": [], "n_active_bh": [], "recycled": [],
    "avg_entropy": [], "influx_rate": [], "w_eff": [],
    "z": [], "mu_simulated": [], "mu_residuals": []
}

print(f"Starting simulation: {N_STEPS} steps")
print(f"Initial scale factor: a={A_INITIAL} (z={1/A_INITIAL - 1:.1f})")
print(f"Parameters: INFLUX_PEAK={INFLUX_PEAK}, IMBALANCE_COUPLING={IMBALANCE_COUPLING}")
print()

for step in range(N_STEPS):
    avg_entropy = compute_avg_entropy()
    added, current_influx_rate = add_new_particles(avg_entropy)
    evolve_particles()
    recycled = absorb_and_evaporate()
    check_and_merge_black_holes()
    try_form_black_hole()
    rho_L_eff, imb, w_proxy = update_cosmology(added, recycled, avg_entropy)
    
    # Redshift: z = 1/a - 1 (valid for a < 1)
    current_z = max(1.0 / a - 1.0, 0.0) if a > 0 else float('inf')
    
    # Record every 120 steps
    if step % 120 == 0:
        n_p = len(particles)
        n_bh = sum(bh["active"] for bh in black_holes)
        history["step"].append(step)
        history["a"].append(a)
        history["H"].append(H)
        history["rho_L_eff"].append(rho_L_eff)
        history["imbalance"].append(imb)
        history["n_particles"].append(n_p)
        history["n_active_bh"].append(n_bh)
        history["recycled"].append(total_recycled)
        history["avg_entropy"].append(avg_entropy)
        history["influx_rate"].append(current_influx_rate)
        history["w_eff"].append(w_proxy)
        history["z"].append(current_z)
        history["mu_simulated"].append(np.nan)  # Will be computed in post-processing
        history["mu_residuals"].append(np.nan)
    
    # Progress report
    if step % 3000 == 0:
        n_p = len(particles)
        n_bh = sum(bh["active"] for bh in black_holes)
        print(f"Step {step:5d}: a={a:.4f}, z={current_z:.3f}, H={H:.6f}, particles={n_p:4d}, BHs={n_bh}")
    
    # Stop if we've reached z=0 (a=1)
    if a >= 1.0:
        print(f"\nReached present day (a=1) at step {step}")
        break

print()
print("Simulation finished.")
print(f"Total instantiated: {total_instantiated}")
print(f"Total recycled: {total_recycled}")
print(f"Total BHs formed: {total_bh_formed}")
print(f"Final scale factor: {a:.4f}")
print(f"Final redshift: {max(1/a - 1, 0):.4f}")

## Post-processing: Normalize distances & compute χ²

In [ ]:
normalize_distance_modulus()
fit_result = compute_chi2_goodness_of_fit()

print("χ² Goodness-of-Fit to Reference Points:")
print(f"  χ²          = {fit_result['chi2']:.3f}")
print(f"  χ²/dof      = {fit_result['chi2_reduced']:.3f}  (dof={fit_result['dof']})")
print(f"  p-value     ≈ {fit_result['p_value']:.4f}")
print(f"  Interpretation: {fit_result['interpretation']}")

## Visualization

In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(14, 16))

# 1. Scale factor a(t)
ax = axs[0, 0]
ax.plot(history["step"], history["a"], 'b-', lw=1.5)
ax.set_xlabel('Step')
ax.set_ylabel('Scale factor a')
ax.set_title('Scale Factor Evolution')
ax.grid(True, alpha=0.3)

# 2. Hubble parameter H(t)
ax = axs[0, 1]
ax.plot(history["step"], history["H"], 'r-', lw=1.5)
ax.set_xlabel('Step')
ax.set_ylabel('H')
ax.set_title('Hubble Parameter')
ax.grid(True, alpha=0.3)

# 3. Effective dark energy density
ax = axs[1, 0]
ax.plot(history["step"], history["rho_L_eff"], 'purple', lw=1.5)
ax.set_xlabel('Step')
ax.set_ylabel('ρ_Λ_eff')
ax.set_title('Effective Dark Energy Density (from Imbalance)')
ax.grid(True, alpha=0.3)

# 4. Equation of state w_eff
ax = axs[1, 1]
ax.plot(history["step"], history["w_eff"], 'green', lw=1.5)
ax.axhline(-1.0, color='black', ls='--', lw=1, label='w = -1 (Λ)')
ax.axhline(-1/3, color='gray', ls=':', lw=1, label='w = -1/3 (accel. threshold)')
ax.set_xlabel('Step')
ax.set_ylabel('w_eff')
ax.set_title('Effective Equation of State')
ax.set_ylim(-2.5, 0)
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)

# 5. Particle count and BH count
ax = axs[2, 0]
ax.plot(history["step"], history["n_particles"], 'C0-', lw=1.5, label='Particles')
ax2 = ax.twinx()
ax2.plot(history["step"], history["n_active_bh"], 'C1-', lw=1.5, label='Active BHs')
ax.set_xlabel('Step')
ax.set_ylabel('Particles', color='C0')
ax2.set_ylabel('Black Holes', color='C1')
ax.set_title('Population Dynamics')
ax.grid(True, alpha=0.3)

# 6. Average entropy and influx rate
ax = axs[2, 1]
ax.plot(history["step"], history["avg_entropy"], 'C2-', lw=1.5, label='Avg Entropy')
ax2 = ax.twinx()
ax2.plot(history["step"], history["influx_rate"], 'C3-', lw=1.5, label='Influx Rate')
ax.set_xlabel('Step')
ax.set_ylabel('Entropy', color='C2')
ax2.set_ylabel('Influx Rate', color='C3')
ax.set_title('Entropy Self-Regulation')
ax.grid(True, alpha=0.3)

# 7. Distance modulus μ(z)
ax = axs[3, 0]
z_valid = [z for z, m in zip(history["z"], history["mu_simulated"]) if m is not None and not np.isnan(m)]
mu_valid = [m for m in history["mu_simulated"] if m is not None and not np.isnan(m)]
if z_valid and mu_valid:
    ax.plot(z_valid, mu_valid, 'C4o-', ms=3, lw=1, label='Simulated μ(z)')
    # Plot reference points
    ax.errorbar(REF_Z_POINTS, REF_MU_VALUES, yerr=REF_SIGMA, fmt='rs', ms=8, 
                capsize=4, label='SNe Ia anchors')
ax.set_xlabel('Redshift z')
ax.set_ylabel('Distance modulus μ')
ax.set_title('Distance Modulus vs Redshift')
ax.legend()
ax.grid(True, alpha=0.3)

# 8. Residuals with χ² annotation
ax = axs[3, 1]
z_res = [z for z, r in zip(history["z"], history["mu_residuals"]) if r is not None and not np.isnan(r)]
res_valid = [r for r in history["mu_residuals"] if r is not None and not np.isnan(r)]
if z_res and res_valid:
    ax.plot(z_res, res_valid, 'C6o-', ms=3, lw=1, label='Δμ residuals')
ax.axhline(0.0, color='black', ls='--', lw=1.5)
ax.set_xlabel('Redshift z')
ax.set_ylabel('Δμ (simulated - ΛCDM)')
ax.set_title('Distance Modulus Residuals')
if fit_result["chi2_reduced"] is not None and not np.isnan(fit_result["chi2_reduced"]):
    txt = f"χ²/dof = {fit_result['chi2_reduced']:.2f}\np ≈ {fit_result['p_value']:.3f}"
    ax.text(0.02, 0.95, txt, transform=ax.transAxes,
            fontsize=10, verticalalignment='top',
            bbox=dict(facecolor='white', alpha=0.8, edgecolor='gray'))
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('ich_simulation_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to: ich_simulation_results.png")

## Summary Statistics

In [ ]:
print("="*60)
print("ICH TOY MODEL - SUMMARY")
print("="*60)
print()
print("COSMOLOGICAL RESULTS:")
print(f"  Final scale factor a      = {history['a'][-1]:.4f}")
print(f"  Final Hubble H            = {history['H'][-1]:.6f}")
print(f"  Final redshift z          = {history['z'][-1]:.4f}")
print(f"  Final w_eff               = {history['w_eff'][-1]:.3f}")
print()
print("CIRCULATION DYNAMICS:")
print(f"  Total instantiated (I∞→AΩ) = {total_instantiated}")
print(f"  Total recycled (AΩ→I∞)     = {total_recycled}")
print(f"  Net imbalance              = {total_instantiated - total_recycled}")
print(f"  Black holes formed         = {total_bh_formed}")
print(f"  Final active BHs           = {history['n_active_bh'][-1]}")
print(f"  Final particles            = {history['n_particles'][-1]}")
print()
print("FIT QUALITY:")
print(f"  χ²          = {fit_result['chi2']:.3f}")
print(f"  χ²/dof      = {fit_result['chi2_reduced']:.3f}")
print(f"  p-value     = {fit_result['p_value']:.4f}")
print(f"  Assessment  : {fit_result['interpretation']}")
print()
print("="*60)